### Load tensorflow

In [1]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [3]:
import pandas as pd
from sklearn import datasets, linear_model

In [4]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [5]:
data_df = pd.read_csv('./prices.csv')
data_df.head(5)

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [6]:
data_df.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [7]:
data_df.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Drop columns `date` and  `symbol`

In [8]:
data_df1 = data_df.drop(['date', 'symbol'], axis = 1)

In [9]:
data_df1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [10]:
data_df1 = data_df1.head(1000)

In [11]:
y = data_df1['close']
data_df1 = data_df1.drop(['close'], axis=1)
X = data_df1

### Divide the data into train and test sets

In [12]:
# Split  training and testing from data
X_train, X_test, y_train, y_test = train_test_split(data_df1, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(800, 4) (800,)
(200, 4) (200,)


## Building the graph in tensorflow

1.Define input data placeholders

In [13]:
# Define Input place holder
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')
x_n = tf.layers.batch_normalization(x, training=True)
#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [14]:
# Define Weights and Bias
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

3.Prediction

In [15]:
# Prediction 
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [16]:
# Loss (Cost) function [Mean square error]

loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [17]:
# GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [18]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [19]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss],feed_dict={x:X_train, 
                                                        y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  8331.386
Training loss at step:  10  is  6787.048
Training loss at step:  20  is  5533.0903
Training loss at step:  30  is  3684.5845
Training loss at step:  40  is  3683.822
Training loss at step:  50  is  3683.4668
Training loss at step:  60  is  3683.3284
Training loss at step:  70  is  3683.2363
Training loss at step:  80  is  3683.186
Training loss at step:  90  is  3683.1855


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [20]:
sess.run(y, feed_dict={x:X_train[0:1]})

array([[68.2832]], dtype=float32)

In [21]:
sess.run(W)
sess.run(b)
print (W.shape, b.shape)

(4, 1) (1,)


In [22]:
_, test_loss = sess.run([train_op, loss],feed_dict={x:X_test, 
                                                        y_:y_test})

### Find the Absolute mean square loss difference between training and testing loss.

In [23]:
Abs_MS_Diff_loss = train_loss - test_loss
Abs_MS_Diff_loss

1394.082

In [24]:
X.shape

(1000, 4)

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [25]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape =(4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(1))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [26]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=30,
          batch_size=X_train.shape[0])

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 1s 937us/step - loss: 8113.0283 - val_loss: 2771.6855
Epoch 2/30
800/800 [==============================] - 0s 5us/step - loss: 6310.5898 - val_loss: 193384.7188
Epoch 3/30
800/800 [==============================] - 0s 6us/step - loss: 312.5311 - val_loss: 365624.8750
Epoch 4/30
800/800 [==============================] - 0s 6us/step - loss: 1511.3359 - val_loss: 1509.2734
Epoch 5/30
800/800 [==============================] - 0s 6us/step - loss: 6888.6450 - val_loss: 19631.8066
Epoch 6/30
800/800 [==============================] - 0s 6us/step - loss: 1769.3243 - val_loss: 309388.8438
Epoch 7/30
800/800 [==============================] - 0s 13us/step - loss: 4739.2769 - val_loss: 108.1544
Epoch 8/30
800/800 [==============================] - 0s 8us/step - loss: 4507.6514 - val_loss: 119041.7188
Epoch 9/30
800/800 [==============================] - 0s 6us/step - loss: 992.2283 - val_loss: 2

### Classification using Keras 

In [27]:
# Load the Iris dataset
iris_data = pd.read_csv('./Iris.csv')
iris_data.head(5)
iris_data = pd.get_dummies(iris_data,prefix=['Species'])
iris_data.head(5)
iris_data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica'],
      dtype='object')

### Load the given Iris data using pandas (Iris.csv)

In [28]:
y = iris_data[['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica']]
X = iris_data.drop(['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica','Id'],axis =1)

### Splitting the data into feature set and target set

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_train:

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
70             5.9           3.2            4.8           1.8
111            6.4           2.7            5.3           1.9
59             5.2           2.7            3.9           1.4
71             6.1           2.8            4.0           1.3
9              4.9           3.1            1.5           0.1
(105, 4)

X_test:

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
55             5.7           2.8            4.5           1.3
145            6.7           3.0            5.2           2.3
92             5.8           2.6            4.0           1.2
10             5.4           3.7            1.5           0.2
120            6.9           3.2            5.7           2.3
(45, 4)


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

### Divide the dataset into Training and test (70:30)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [30]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape = (4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

### Fitting the model and predicting 

In [31]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='relu'))

In [32]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='relu'))

In [33]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [34]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 4)                 16        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                40        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 27        
Total params: 186
Trainable params: 178
Non-trainable params: 8
_________________________________________________________________


In [36]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=30,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/30
105/105 [==============================] - 1s 13ms/step - loss: 0.9454 - acc: 0.6000 - val_loss: 1.3239 - val_acc: 0.2889
Epoch 2/30
105/105 [==============================] - 0s 172us/step - loss: 0.8978 - acc: 0.6667 - val_loss: 1.3281 - val_acc: 0.2889
Epoch 3/30
105/105 [==============================] - 0s 250us/step - loss: 0.8670 - acc: 0.6952 - val_loss: 1.3388 - val_acc: 0.2889
Epoch 4/30
105/105 [==============================] - 0s 226us/step - loss: 0.8249 - acc: 0.7048 - val_loss: 1.3330 - val_acc: 0.2889
Epoch 5/30
105/105 [==============================] - 0s 207us/step - loss: 0.7873 - acc: 0.7429 - val_loss: 1.3181 - val_acc: 0.2889
Epoch 6/30
105/105 [==============================] - 0s 246us/step - loss: 0.7804 - acc: 0.7333 - val_loss: 1.3177 - val_acc: 0.2889
Epoch 7/30
105/105 [==============================] - 0s 409us/step - loss: 0.7034 - acc: 0.7810 - val_loss: 1.3023 - val_acc: 0.2889
Epoch 8/30
105/105

### Report Accuracy of the predicted values

In [37]:
scores = model.evaluate(X_test,y_test)
scores

45/45 [==============================] - 0s 178us/step


[0.5693430198563469, 0.6666666719648573]